In [34]:
import copy
import numpy
import random
import math
import matplotlib.pyplot as plt
import pandas as pd
from collections import deque

In [35]:
#Parameters
GROUP_ID = 'Group04'
ALGORITHM = 'bt'
PUZZLE_TYPE = 'easy'
PUZZLE_PATH = 'puzzles/Easy-P1.txt'

In [36]:
#class for puzzle tiles
class sudokuTile:
    def __init__(self, pos, val, layer):
        self.position = pos
        self.layer = layer
        self.value = val
        self.fixed = True
        self.row = pos[0]
        self.col = pos[1]
        if val == '?':
            self.fixed = False
            self.domain = ['1','2','3','4','5','6','7','8','9']
        else:
            self.domain = [val]

In [37]:
 #Processes the sudoku file into a numpy array of sudokuTile objects
def processPuzzle(filePath):
    arr = []
    layer = 0
    with open(filePath, 'r') as file:
        firstLine = True
        rowNum = 0
        for line in file:
            colNum = 0
            processedLine = line.rstrip('\n')
            if(firstLine):
                #the first line of each text file had some (corrupted?) characters at the start, so the first 3 characters had to be cut off-- may need to be changed depending on text files
                row = processedLine[0:].split(",")
                #print(line)
                firstLine = False
            else:
                row = processedLine.split(",")
            tileRow = []
            for num in row:
                tile = sudokuTile([rowNum, colNum], num, layer)
                layer += 1
                tileRow.append(tile)
                colNum += 1
            arr.append(tileRow)
            rowNum += 1
        file.close()
    puzzle = numpy.array(arr)
    return puzzle

#iterates over the given puzzle and prints the value of each tile
def printPuzzle(puzzle):
    for row in range(9):
        for column in range(8):
            print(puzzle[row][column].value + ",", end = "")
        print(puzzle[row][8].value)
    print("\n")

#returns a list of all tiles constrained by a tile in the given position
def getConstrainedTiles(puzzle, position):
    row = position[0]
    col = position[1]
    positionsConstrained = set()
    #gets all the positions in the same row
    for c in range(9):
        if(c != col):
            positionsConstrained.add((row,c))
    #gets all the positions in the same column
    for r in range(9):
        if(r != row):
            positionsConstrained.add((r,col))
    #gets all the positions in the same 3x3 grid
    box_row = (row // 3) * 3
    box_col = (col // 3) * 3
    for r in range(box_row, box_row + 3):
        for c in range(box_col, box_col + 3):
            if (r != row and c != col):
                positionsConstrained.add((r,c))
    tiles = []
    #returns all the tiles associated with the collected positions
    for pos in positionsConstrained:
        tiles.append(puzzle[pos[0]][pos[1]])
    return tiles

#custom puzzle copier for increased efficiency in runtime
def copyPuzzle(puzzle):
    arr = []
    for row in range(9):
        rowArr = []
        for col in range(9):
            originalTile = puzzle[row][col]
            tile = sudokuTile([row, col], originalTile.value, originalTile.layer)
            tile.domain = originalTile.domain
            rowArr.append(tile)
        arr.append(rowArr)
    newPuzzle = numpy.array(arr)
    return newPuzzle

# Fill in the blank spots in the puzzle to create a random puzzle.
# Fill in 3x3 squares at a time such that all values in the square are different
def generateRandomPuzzle(originalPuzzle):
    newPuzzle = copy.deepcopy(originalPuzzle)
    for i in range(3):
        for j in range(3):
            nums = ['1','2','3','4','5','6','7','8','9']
            #checks each 3x3 grid for set tiles and removes their value from the list of possible nums
            for row in range(0+(i*3), (i*3)+3):
                for column in range(0+(j*3), (j*3)+3):
                    tile = originalPuzzle[row][column]
                    if (tile.fixed == True):
                        nums.remove(tile.value)
            #randomly sets each unfixed tile in the grid to a random (unpicked) num
            for row in range(0+(i*3), (i*3)+3):
                for column in range(0+(j*3), (j*3)+3):
                    tile = newPuzzle[row][column]
                    if not(tile.fixed):
                        tile.value = random.choice(nums)
                        nums.remove(tile.value)
    return newPuzzle


In [38]:
#adds up all the constraints violated in the puzzle and returns the value
def checkConstraintsViolated(p):
    violations = 0
    # checks if puzzle passes row rule
    for row in range(9):
        nums = []
        for column in range(9):
            nums.append(p[row][column].value)
        newNums = [item for item in nums if item != "?"]
        setNums = set(newNums)
        violations += len(newNums)- len(setNums)

    # checks if puzzle passes column rule
    for column in range(9):
        nums = []
        for row in range(9):
            nums.append(p[row][column].value)
        newNums = [item for item in nums if item != "?"]
        setNums = set(newNums)
        violations += len(newNums)- len(setNums)

    # checks if puzzle passes 3x3 grid rules
    for i in range(3):
        for j in range(3):
            nums = []
            for row in range(0 + (i * 3), (i * 3) + 3):
                for column in range(0 + (j * 3), (j * 3) + 3):
                    nums.append(p[row][column].value)
            newNums = [item for item in nums if item != "?"]
            setNums = set(newNums)
            violations += len(newNums)- len(setNums)
    return violations

#checks the validity of the puzzle based on if any constraints were violated
def checkPuzzle(p):
    numConstraintsViolated = checkConstraintsViolated(p)
    if(numConstraintsViolated > 0):
        return False
    else:
        return True

In [39]:
#Simple backtracking algorithm using a stack
def backTracking(puzzle):
    #setting up the stack with the inital possible values for the first tile
    stack = [(puzzle[0][0], d) for d in puzzle[0][0].domain]
    layer = 0
    steps = 0
    while len(stack) > 0:
        # pop the top value off of the stack, and set the given tile's value to the given value
        prevLayer = layer
        steps += 1
        action = stack.pop()
        tile = action[0]
        val = action[1]
        layer = tile.layer
        tile.value = val
        # when backtracking, cleanup all unfixed tiles by turning them back into unknowns
        if (prevLayer > tile.layer):
            for i in range(layer + 1, prevLayer):
                row = i // 9
                col = i % 9
                resetTile = puzzle[row][col]
                if not (resetTile.fixed):
                    resetTile.value = '?'

        # if the value is valid for the puzzle, add all the possible values for the next position on the board to the stack
        if (checkPuzzle(puzzle)):
            layer = tile.layer + 1
            # puzzle has been solved
            if (layer == 81):
                break
            row = layer // 9
            col = layer % 9
            nextTile = puzzle[row][col]
            #add the next transitions to the stack
            for d1 in nextTile.domain:
                stack.append((nextTile, d1))
        # if the value is not valid, set the tile back to an unknown, and don't continue searching that path (prune said branch)
        else:
            layer = tile.layer
            if (not tile.fixed):
                tile.value = '?'

    return puzzle, steps

In [40]:
#Backtracking algorithm with one-step forward checking
def forwardChecking(puzzle):
    #setting up the stack with the inital possible values for the first tile
    #forward checking also needs to store the state of the puzzle due to edited domains
    stack = [(puzzle[0][0], d, puzzle) for d in puzzle[0][0].domain]
    layer = 0
    steps = 0
    while len(stack) > 0:
        # pop the top value off of the stack, and set the given tile's value to the given value
        prevLayer = layer
        steps += 1
        action = stack.pop()
        stackPuzzle = copyPuzzle(action[2])
        position = action[0].position
        tile = stackPuzzle[position[0],position[1]]
        layer = tile.layer
        val = action[1]
        tile.value = val
        domainsGood = True
        constrainedTiles = getConstrainedTiles(stackPuzzle, tile.position)
        #checks the domains of each tile constrained by the tile being changed
        for tileEdited in constrainedTiles:
            checkedDomain = [num for num in tileEdited.domain if num != val]
            if(len(checkedDomain) == 0):
                domainsGood = False
        #If no domains had a length of 0, edit domains and add all the possible values for the next position on the board to the stack
        if (domainsGood):
            #actually editing domains of constrained tiles
            for tileEdited in constrainedTiles:
                tileEdited.domain = [num for num in tileEdited.domain if num != val]
            layer = tile.layer + 1
            #case where puzzle has been solved
            if (layer == 81):
                break
            row = layer // 9
            col = layer % 9
            nextTile = stackPuzzle[row][col]
            #Gets all the tiles constrained by the current tile and checks their edited domains
            for d1 in nextTile.domain:
                stack.append((nextTile, d1, stackPuzzle))
        # if the value is not valid, set the tile back to an unknown, and don't continue searching that path (prune said branch)
        else:
            layer = tile.layer
            if (not tile.fixed):
                tile.value = '?'

    puzzle = stackPuzzle
    return puzzle, steps

In [41]:
#class to store arcs between tiles
class arc:
    def __init__(self,tile1,tile2):
        self.t1 = tile1
        self.t2 = tile2

#reduces the domains of the tiles in the given puzzle through AC-3
def arcConsistency(puzzle):
    queue = addInitialArcs(puzzle)
    #stores arcs in a double sided queue
    while (len(queue)!= 0):
        currentArc = queue.popleft()
        if removeInconsistentValues(currentArc):
            # Check if first tiles domain is empty
            if not(currentArc.t1.domain):
                return False, puzzle
            neighborArcs = addNeighborArcsExcept(currentArc,puzzle)
            queue.extend(neighborArcs)
    return True, puzzle

#adds all the initial arcs making up the starting puzzle to the queue
def addInitialArcs(puzzle):
    queue = deque()
    # Add row arcs
    for row in range(9):
        for i in range(8):
            for j in range(i+1,9):
                newArc = arc(puzzle[row][i],puzzle[row][j])
                queue.append(newArc)

    # Add column arcs
    for col in range(9):
        for i in range(8):
            for j in range(i+1,9):
                newArc = arc(puzzle[i][col],puzzle[j][col])
                queue.append(newArc)

    #add subgrid arcs
    for row in [0,4,8]:
        for col in [0,4,8]:
            box_row = (row // 3) * 3
            box_col = (col // 3) * 3
            for r in range(box_row, box_row + 3):
                for c in range(box_col, box_col + 3):
                    for i in range(box_row, box_row + 3):
                        for j in range(box_col, box_col + 3):
                            if (i, j) > (r, c):
                                newArc = arc(puzzle[r][c], puzzle[i][j])
                                queue.append(newArc)
    return queue

#adds neighboring arcs given an arc
def addNeighborArcsExcept(currentArc,puzzle):
    tile1 = currentArc.t1
    tile2 = currentArc.t2
    queue = deque()
    # Add row arcs
    for i in range(9):
        #makes sure the same row arc isn't added again
        if (i != tile1.col and puzzle[tile1.row][i] != tile2):
            newArc = arc(puzzle[tile1.row][i],tile1)
            queue.append(newArc)

    # Add col arcs
    for i in range(9):
        #makes sure the same column arc isn't added again
        if (i != tile1.row and puzzle[i][tile1.col] != tile2):
            newArc = arc(puzzle[i][tile1.col],tile1)
            queue.append(newArc)


    #add subgrid arcs
    for row in [0,4,8]:
        for col in [0,4,8]:
            box_row = (row // 3) * 3
            box_col = (col // 3) * 3
            for r in range(box_row, box_row + 3):
                for c in range(box_col, box_col + 3):
                    for i in range(box_row, box_row + 3):
                        for j in range(box_col, box_col + 3):
                            if (i, j) > (r, c):
                                #makes sure the same subgrid arc isn't added again
                                if(tile2 != puzzle[i][j]):
                                    newArc = arc(puzzle[r][c], puzzle[i][j])
                                    queue.append(newArc)
    return queue

#removes all values from a tile's domains in an arc that would conflict with the other tile in the arc
def removeInconsistentValues(currentArc):
    removed = False
    tile1 = currentArc.t1
    tile2 = currentArc.t2
    for value1 in tile1.domain:
        isDiff = False
        for value2 in tile2.domain:
            if(value2 != value1):
                isDiff = True
                break
        if not(isDiff):
            tile1.domain.remove(value1)
            removed = True
    return removed

In [42]:
# Makes a new puzzle from a given puzzle by swapping two values within one of the 3x3 grids
def getNeighborPuzzle(puzzle):
    neighbor = copy.deepcopy(puzzle)
    startrow = random.choice([0,3,6])
    startcol = random.choice([0,3,6])
    fixed = 0
    for r in range(startrow, startrow + 3):
        for c in range(startcol, startcol + 3):
            if (neighbor[r][c].fixed ==True):
                fixed = fixed + 1
    # Check if 3x3 square has all fixed points
    if fixed >= 8:
        return neighbor
    row1 = random.choice(range(startrow,startrow + 3))
    col1 = random.choice(range(startcol,startcol + 3))
    row2 = random.choice(range(startrow,startrow + 3))
    col2 = random.choice(range(startcol,startcol + 3))

    while(neighbor[row1][col1].fixed == True):
        row1 = random.choice(range(startrow,startrow + 3))
        col1 = random.choice(range(startcol,startcol + 3))
    while(neighbor[row2][col2].fixed == True):
        row2 = random.choice(range(startrow,startrow + 3))
        col2 = random.choice(range(startcol,startcol + 3))

    temp = neighbor[row1][col1]
    neighbor[row1][col1] = neighbor[row2][col2]
    neighbor[row2][col2] = temp
    return neighbor

def simulatedAnnealing(puzzle,maxSteps,temp,a):
    currentPuzzle = generateRandomPuzzle(puzzle)
    k = 1
    cutoff = 1e-4
    for iteration in range(maxSteps):
        t = temp/((iteration+1)*a)
        if (checkConstraintsViolated(currentPuzzle) == 0):
            return currentPuzzle, 0, iteration
        if (t < cutoff):
            return currentPuzzle, checkConstraintsViolated(currentPuzzle), iteration
        nextPuzzle = getNeighborPuzzle(currentPuzzle)
        delta = checkConstraintsViolated(currentPuzzle) - checkConstraintsViolated(nextPuzzle)

        #if the generated puzzle is better, use that puzzle
        if (delta > 0):
            currentPuzzle = nextPuzzle

        #if the generated puzzle is worse, use it depending on a random value and the current temp
        elif (random.random() > math.exp(delta/k*t)):
            currentPuzzle = nextPuzzle

    return currentPuzzle, checkConstraintsViolated(currentPuzzle), iteration


In [43]:
#multipoint crossover by randomly choosing a parent for each 3x3 subgrid
def geneticCrossover(firstPuzzle, secondPuzzle):
    newPuzzle = copy.deepcopy(firstPuzzle)
    #chooses a random subgrid
    for row in range(0, 7, 3):
        for col in range(0,7,3):
            randomNum = random.uniform(0,1)
            #if the random number is less than 0.5 (coin flip), switch all tiles of the 3x3 grid
            if (randomNum < 0.5):
                for r in range(row, row+3):
                    for c in range(col,col+3):
                        newPuzzle[r][c].value = secondPuzzle[r][c].value
    return newPuzzle

#mutate by picking a random 3x3 grid and swapping two (unfixed) values
def geneticMutation(mutationRate, puzzle):
    newPuzzle = copy.deepcopy(puzzle)
    randomNum = random.uniform(0,1)
    if (randomNum <= mutationRate):
        startrow = random.choice([0,3,6])
        startcol = random.choice([0,3,6])
        row1 = random.choice(range(startrow,startrow + 3))
        col1 = random.choice(range(startcol,startcol + 3))
        row2 = random.choice(range(startrow,startrow + 3))
        col2 = random.choice(range(startcol,startcol + 3))
        while(newPuzzle[row1][col1].fixed == True):
            row1 = random.choice(range(startrow,startrow + 3))
            col1 = random.choice(range(startcol,startcol + 3))
        while(newPuzzle[row2][col2].fixed == True):
            row2 = random.choice(range(startrow,startrow + 3))
            col2 = random.choice(range(startcol,startcol + 3))
        tile1 = newPuzzle[row1][col1]
        tile2 = newPuzzle[row2][col2]
        temp = tile1.value
        tile1.value = tile2.value
        tile2.value = temp
    return newPuzzle

def geneticAlgorithm(originalPuzzle, popSize, mutationRate, epochs, tourneySize):
    population = {}
    #initializing the population
    avgFit = 0
    for i in range(popSize):
        #create new puzzle
        randomPuzzle = generateRandomPuzzle(originalPuzzle)
        randomPuzzle = tuple(map(tuple, randomPuzzle))
        #calculate the fitness of the puzzle
        fitness = 1/(checkConstraintsViolated(randomPuzzle))
        avgFit += 1/fitness
        population[randomPuzzle] = fitness

    #calculate avg fitness of population and best fitness for metrics
    constraintsViolated = [1/max(population.values())]
    avgFit /= popSize
    constraintsViolated2 = [avgFit]
    iterations = [0]

    #number of epochs
    for i in range(epochs):
        avgFit = 0
        newPopulation = {}
        for puzzle in population:
            selectionPopulation = copy.deepcopy(population)
            #TOURNAMENT:: MAY THE 2 BEST WIN
            tournament = {}
            for t in range(tourneySize):
                randomChamp = random.choice(list(selectionPopulation.keys()))
                tournament[randomChamp] = selectionPopulation[randomChamp]
                selectionPopulation.pop(randomChamp)

            #crossover and mutation for creating a new individual given 2 best parents
            firstParent = max(tournament.keys(), key=tournament.get)
            tournament.pop(firstParent)
            secondParent = max(tournament.keys(), key=tournament.get)
            newPuzzle = geneticCrossover(firstParent, secondParent)
            newPuzzle = geneticMutation(mutationRate, newPuzzle)

            #need to calculate loss of new puzzle again and add it to the new population
            c = checkConstraintsViolated(newPuzzle)
            #if there are no violated constraints, the solution has been found
            if(c == 0):
                return newPuzzle, 0, i
            fitness = 1/c
            avgFit += 1/fitness
            newPuzzle = tuple(map(tuple, newPuzzle))
            newPopulation[newPuzzle] = fitness

        population = newPopulation
        #calculate avg fitness of population and best fitness for metrics
        best = max(population, key = population.get)
        constraintsViolated.append(checkConstraintsViolated(best))
        print(constraintsViolated)
        avgFit /= popSize
        constraintsViolated2.append(avgFit)
        iterations.append(i+1)
    #Graphing functions
    # plt.plot(iterations,constraintsViolated, label = 'Best Individual')
    # plt.plot(iterations,constraintsViolated2, label = 'Population Average')
    # plt.xlabel("Iterations of Population")
    # plt.ylabel("Number of violations")
    # plt.title("Constraint Violations VS. Iteration (Medium example)")
    # plt.legend()
    # plt.savefig('MediumPuzzleGA.png')
    # plt.show()
    finalPuzzle = max(population.keys(), key=population.get)
    return finalPuzzle, checkConstraintsViolated(finalPuzzle), i

In [44]:
#main driver class to solve puzzles given a puzzle file and an algorithm
puzzle = processPuzzle(PUZZLE_PATH)
originalPuzzle = copy.deepcopy(puzzle)
print(ALGORITHM)
print(PUZZLE_PATH)
print("=====================")
print("Original Puzzle: ")
printPuzzle(originalPuzzle)
#Solve the puzzle based on the given algorithm
if(ALGORITHM == 'bt'):
    solvedPuzzle, steps = backTracking(puzzle)
    print("Solved Puzzle in", steps, "steps:")
elif(ALGORITHM == 'fc'):
    solvedPuzzle, steps = forwardChecking(puzzle)
    print("Solved Puzzle in", steps, "steps:")
elif(ALGORITHM == 'ac3'):
    consistent, newPuzzle = arcConsistency(puzzle)
    if(consistent):
        solvedPuzzle, steps = backTracking(newPuzzle)
        print("Solved Puzzle in", steps, "steps:")
    else:
        print("Given Puzzle couldn't be made consistent")
elif(ALGORITHM == 'sa'):
    solvedPuzzle, violations, iterations = simulatedAnnealing(originalPuzzle,100000,10,.7)
    print("Best Puzzle in", iterations, "iterations with", violations, "constraint violations")
elif(ALGORITHM == 'ga'):
    solvedPuzzle, violations, generations = geneticAlgorithm(originalPuzzle, 150, .1, 100, 5)
    print("Best Puzzle in", generations, "generations with", violations, "constraint violations")
else:
    print("Algorithm parameter not recognized")
printPuzzle(solvedPuzzle)


#Writing the finished puzzle to a file
fileName = GROUP_ID + '_' + ALGORITHM + "_" + PUZZLE_TYPE + "_" + PUZZLE_PATH.rstrip('.txt')[-2:]
print(fileName)
with open(fileName, 'w') as file:
    for row in range(9):
        for column in range(8):
            file.write(solvedPuzzle[row][column].value + ",")
        file.write(solvedPuzzle[row][8].value + '\n')
    file.close()


bt
puzzles/Easy-P1.txt
Original Puzzle: 
?,?,8,?,5,6,?,?,?
7,?,4,?,?,?,6,1,9
?,?,?,?,?,?,8,5,?
6,?,7,?,2,9,5,?,?
?,?,9,?,6,?,1,?,?
?,?,2,3,1,?,9,?,4
?,3,5,?,?,?,?,?,?
4,2,1,?,?,?,3,?,6
?,?,?,8,3,?,4,?,?


Solved Puzzle in 8117 steps:
1,9,8,7,5,6,2,4,3
7,5,4,2,8,3,6,1,9
2,6,3,1,9,4,8,5,7
6,1,7,4,2,9,5,3,8
3,4,9,5,6,8,1,7,2
5,8,2,3,1,7,9,6,4
8,3,5,6,4,2,7,9,1
4,2,1,9,7,5,3,8,6
9,7,6,8,3,1,4,2,5


Group04_bt_easy_P1
